# AIAK训练llama3-70b最佳实践

本方案旨在帮助大模型开发者快速上手在百舸平台使用AIAK进行大模型训练，以llama3-70b为例演示如何实现大模型训练。

## 步骤一：准备模型权重

### 下载权重

1. 从ModelScope社区下载权重，安装下载工具

In [14]:
! pip install modelscope

  Using cached modelscope-1.18.0-py3-none-any.whl.metadata (40 kB)
   ━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.3/5.7 MB 38.8 kB/s eta 0:02:20m
ERROR: Exception:
Traceback (most recent call last):
  File "/root/anaconda3/lib/python3.11/site-packages/pip/_vendor/urllib3/response.py", line 438, in _error_catcher
    yield
  File "/root/anaconda3/lib/python3.11/site-packages/pip/_vendor/urllib3/response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "/root/anaconda3/lib/python3.11/site-packages/pip/_vendor/urllib3/response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "/root/anaconda3/lib/python3.11/site-packages/pip/_vendor/cachecontrol/filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
                  ^^^^^^^^^^^^^^^^^^^
  File "/root/anaconda3/lib/python3.11/http/client.py", line 473, in read
    s = self.fp

2. 下载原始权重到PFS存储

In [12]:
! modelscope download --model LLM-Research/Meta-Llama-3-70B --local_dir '/root/pfs/models/llama3-70b/modelscope.cn/models/LLM-Research/Meta-Llama-3-70B'

/usr/bin/sh: 1: modelscope: not found


### 权重格式转换
权重格式转换，将权重从HF格式转换为MegatronCore格式，提交一个权重转换任务

In [ ]:
! pip install python-dotenv
! pip install bce-python-sdk-next==100.9.19.2
! pip install future==1.0.0
! pip install pycryptodome==3.20.0

In [ ]:
# -*- coding: utf-8 -*-
import sys
import json
import os
import time
import logging
from baidubce.services.aihc.aihc_client import AIHCClient

import baidubce.protocol
from baidubce.bce_client_configuration import BceClientConfiguration
from baidubce.auth.bce_credentials import BceCredentials

from dotenv import load_dotenv
import os

# 加载.env文件
load_dotenv()

# 获取配置信息, 从环境变量中获取,需要先在.env文件中设置环境变量
ak = os.getenv("AK")
sk = os.getenv("SK")
host=os.getenv("HOST")

client_token = 'test-aihc-' + str(int(time.time()))
logging.info('client_token: %s', client_token)

config = BceClientConfiguration(
    credentials=BceCredentials(ak, sk),
    endpoint=host,
    protocol=baidubce.protocol.HTTPS
)

aihc_client = AIHCClient(config)

# 创建任务
resourcePoolId = 'your resourcePoolId'
payload = {
    "queue": "default",
    "priority": "normal",
    "jobFramework": "PyTorchJob",
    "name": "llama2-70b-ck-quickstart-v1",
    "jobSpec": {
        "command": "bash /workspace/AIAK-Training-LLM/examples/llama3/checkpoint_convert/convert_llama3_70b_hf_to_mcore.sh",
        "image": "registry.baidubce.com/aihc-aiak/aiak-training-llm:ubuntu22.04-cu12.3-torch2.2.0-py310-bccl1.2.7.2_v2.1.1.5_release",
        "replicas": 1,
        "envs": [
            {
                "name":"LOAD",
                "value":"/root/pfs/models/llama3-70b/modelscope.cn/models/LLM-Research/Meta-Llama-3-70B"
            },
                        {
                "name":"SAVE",
                "value":"/root/pfs/models/llama3-70b/mcore/modelscope.cn/models/LLM-Research/Meta-Llama-3-70B/tp4_pp4"
            }
        ]
    },
    "datasources": [
        {
            "type": "pfs",
            "name": "pfs-oYQuh4",
            "mountPath": "/root/pfs"
        }
    ]
}
res = aihc_client.create_aijob(
    client_token=client_token,
    resourcePoolId=resourcePoolId,
    payload=payload
)
print('res', res)

## 步骤二：准备预训练数据

### 下载数据集

In [6]:
! wget -o /root/pfs/datasets/WuDaoCorpus2.0_base_sample.tgz https://cce-ai-datasets.cdn.bcebos.com/datasets/aiak/WuDaoCorpus2.0_base_sample.tgz
! tar zxvf WuDaoCorpus2.0_base_sample.tgz -C /root/pfs/datasets

WuDaoCorpus2.0_base_sample/
WuDaoCorpus2.0_base_sample/part-1000.json
WuDaoCorpus2.0_base_sample/part-4000.json
WuDaoCorpus2.0_base_sample/part-6000.json
WuDaoCorpus2.0_base_sample/part-3000.json
WuDaoCorpus2.0_base_sample/part-2000.json
WuDaoCorpus2.0_base_sample/part-9000.json
WuDaoCorpus2.0_base_sample/part-7000.json
WuDaoCorpus2.0_base_sample/part-5000.json
WuDaoCorpus2.0_base_sample/part-8000.json
WuDaoCorpus2.0_base_sample/part-0.json


In [ ]:
### 数据格式转换

解压缩后得到10个JSON文件，文件内容为数组对象格式：

```
[
    {
        "id": 1,
        "uniqueKey": "da73e2d0bb4e39d241c3806876621da7",
        "titleUkey": "da73e2d0bb4e39d241c3806876621da7",
        "dataType": "博客",
        "title": "引网站蜘蛛的方法",
        "content": "做站长的都希望自已做的网站被搜索引擎比如百度尽早收录。对于新站来说,蜘蛛可没不是呼之即来挥之即去的。 但是也不是一筹莫展,无计可施,只有摸透了这一只只神秘莫测的蜘蛛,有的放矢,才能随心所欲,对吧。呵呵。些话不多说,言归正传吧。 第一,蜘蛛的出动其实是非常讲究效率的,他们也懒得白跑,如果你的网站十天半个月不更新,他白跑几次后,也就不会来这么勤了。 所以,为了让蜘蛛天天来,那么就务必不让他空来,每次都喂点食。所以对策说就是最好每天更新内容了。 可以说,你规律性的多久更新一次,蜘蛛很可能也多久才来一次。 第二,尽量去掉网页上可有可无的部分吧,特别是java之类的,还有过大的图片,要尽量降低网页加载负荷,,加速网页的打开速度,网页打速度快,那么用户体验才好,跳出率才低,网页评分才高。 第三,检查内部链接结构,去除死链接和重复链接；死链接让蜘蛛原地打转,重复连接降低网页的新鲜度； 第四,尽量多从正规和相关站点获得反向链接,正规的链接能确保外链的稳定,以及免收株连；相关链接能提高外链的权重； 第五,为站点制作网站地图,包括格式和xml两种格式,作为蜘蛛爬行的向导,让蜘蛛能爬满整个网站而没有遗漏； 第六,确保服务器返回正确的数据包响应,这条比较玄,还不懂,你可以跟我说下什么意思； 第七、为每个页面制作独立的标题和meta标签(关键字、描述),这个在网页模板里写好调用代码就行； 第八、查看网页日志,监测蜘蛛的的爬行记录,蜘蛛爬行后会留下足迹,查看这些足迹就知道蜘蛛什么时候曾经光顾过这里了。 第九,直接快速的方法:用繁殖池自动繁殖引蜘蛛,它可以快速收录站群链接或者外推链接,可以实现亿万级蜘蛛网互联互串引蜘蛛,可以增加网站收录,提升网站排名。使用繁殖池引蜘蛛到其他平台发布外链不会受到种种限制。 需要繁殖池的联系官方qq: 咨询,马上为您申请开通！"
    },
    {
        "id": 2,
        "uniqueKey": "0d78638721aecbdd798a072746114d20",
        "titleUkey": "0d78638721aecbdd798a072746114d20",
        "dataType": "博客",
        "title": "千站云繁殖池收录神器是什么?",
        "content": "千站云繁殖池是一款全新的升级收录算法,全面升级,6大功能,快速收录站群链接或者外推链接,已经取代于蜘蛛池,蜘蛛池的效果差已过时了； 那什么是繁殖池？适用于什么？有什么作用？ 繁殖池是大量网站将百度蜘蛛来访时集成一个池,通过程序控制自动繁殖外链地址给蜘蛛,这样可以快速大量收录站群链接或者外推链接； 适用于医疗媒体外推、站群、泛站、目录群、寄生虫、博客、微博、论坛、b2b信息,全自动繁殖不同地址引蜘蛛,实用而操作简单,效果好,可以实现亿万级蜘蛛网互联互串引蜘蛛,可以让新站、外推链接、媒体链接等等快速增加收录,被k网站也可以尽快恢复权重和搜索引擎快照,正常收录的网站可以增加网站收录,提升网站排名让你感受到不再需要为引蜘蛛到其他平台发布外链而受到种种限制等欢喜； 如果需要可以联系官方qq: 马上申请为你开通！！！"
    }
]
```

需要先将其处理为每行一个完整对象的jsonl格式：

```
{"id":1,"uniqueKey":"da73e2d0bb4e39d241c3806876621da7","titleUkey":"da73e2d0bb4e39d241c3806876621da7","dataType":"博客","title":"引网站蜘蛛的方法","content":"做站长的都希望自已做的网站被搜索引擎比如百度尽早收录。对于新站来说,蜘蛛可没不是呼之即来挥之即去的。 但是也不是一筹莫展,无计可施,只有摸透了这一只只神秘莫测的蜘蛛,有的放矢,才能随心所欲,对吧。呵呵。些话不多说,言归正传吧。 第一,蜘蛛的出动其实是非常讲究效率的,他们也懒得白跑,如果你的网站十天半个月不更新,他白跑几次后,也就不会来这么勤了。 所以,为了让蜘蛛天天来,那么就务必不让他空来,每次都喂点食。所以对策说就是最好每天更新内容了。 可以说,你规律性的多久更新一次,蜘蛛很可能也多久才来一次。 第二,尽量去掉网页上可有可无的部分吧,特别是java之类的,还有过大的图片,要尽量降低网页加载负荷,,加速网页的打开速度,网页打速度快,那么用户体验才好,跳出率才低,网页评分才高。 第三,检查内部链接结构,去除死链接和重复链接；死链接让蜘蛛原地打转,重复连接降低网页的新鲜度； 第四,尽量多从正规和相关站点获得反向链接,正规的链接能确保外链的稳定,以及免收株连；相关链接能提高外链的权重； 第五,为站点制作网站地图,包括格式和xml两种格式,作为蜘蛛爬行的向导,让蜘蛛能爬满整个网站而没有遗漏； 第六,确保服务器返回正确的数据包响应,这条比较玄,还不懂,你可以跟我说下什么意思； 第七、为每个页面制作独立的标题和meta标签(关键字、描述),这个在网页模板里写好调用代码就行； 第八、查看网页日志,监测蜘蛛的的爬行记录,蜘蛛爬行后会留下足迹,查看这些足迹就知道蜘蛛什么时候曾经光顾过这里了。 第九,直接快速的方法:用繁殖池自动繁殖引蜘蛛,它可以快速收录站群链接或者外推链接,可以实现亿万级蜘蛛网互联互串引蜘蛛,可以增加网站收录,提升网站排名。使用繁殖池引蜘蛛到其他平台发布外链不会受到种种限制。 需要繁殖池的联系官方qq: 咨询,马上为您申请开通！"}
{"id":2,"uniqueKey":"0d78638721aecbdd798a072746114d20","titleUkey":"0d78638721aecbdd798a072746114d20","dataType":"博客","title":"千站云繁殖池收录神器是什么?","content":"千站云繁殖池是一款全新的升级收录算法,全面升级,6大功能,快速收录站群链接或者外推链接,已经取代于蜘蛛池,蜘蛛池的效果差已过时了； 那什么是繁殖池？适用于什么？有什么作用？ 繁殖池是大量网站将百度蜘蛛来访时集成一个池,通过程序控制自动繁殖外链地址给蜘蛛,这样可以快速大量收录站群链接或者外推链接； 适用于医疗媒体外推、站群、泛站、目录群、寄生虫、博客、微博、论坛、b2b信息,全自动繁殖不同地址引蜘蛛,实用而操作简单,效果好,可以实现亿万级蜘蛛网互联互串引蜘蛛,可以让新站、外推链接、媒体链接等等快速增加收录,被k网站也可以尽快恢复权重和搜索引擎快照,正常收录的网站可以增加网站收录,提升网站排名让你感受到不再需要为引蜘蛛到其他平台发布外链而受到种种限制等欢喜； 如果需要可以联系官方qq: 马上申请为你开通！！！"}
```

In [1]:
import json
import os

# 定义要保存的jsonl文件名
output_file = '/root/pfs/datasets/WuDaoCorpus2.0_base_sample.jsonl'
input_dir = '/root/pfs/datasets/WuDaoCorpus2.0_base_sample'
# 打开文件准备写入
with open(output_file, 'w') as file:
    # 遍历当前目录下所有的.json文件
    for filename in os.listdir(input_dir):
        if filename.endswith('.json'):
            print(filename)
            # 读取json文件内容
            with open(input_dir+ '/' +filename, 'r') as json_file:
                data_list = json.load(json_file)
                # 遍历数据列表
                for data in data_list:
                    # 将字典转换为JSON字符串
                    data['text'] = data['content']
                    data['content'] = ''
                    json_str = json.dumps(data)
                    print(json_str)
                    # 写入文件，并添加换行符
                    file.write(json_str + '\n')
print(output_file)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



### 数据预处理

In [ ]:
# -*- coding: utf-8 -*-
import sys
import json
import os
import time
import logging
from baidubce.services.aihc.aihc_client import AIHCClient

import baidubce.protocol
from baidubce.bce_client_configuration import BceClientConfiguration
from baidubce.auth.bce_credentials import BceCredentials

from dotenv import load_dotenv
import os

# 加载.env文件
load_dotenv()

# 获取配置信息, 从环境变量中获取,需要先在.env文件中设置环境变量
ak = os.getenv("AK")
sk = os.getenv("SK")
host=os.getenv("HOST")

client_token = 'test-aihc-' + str(int(time.time()))
logging.info('client_token: %s', client_token)

config = BceClientConfiguration(
    credentials=BceCredentials(ak, sk),
    endpoint=host,
    protocol=baidubce.protocol.HTTPS
)

aihc_client = AIHCClient(config)

# 创建任务
resourcePoolId = 'your resourcePoolId'
payload = {
    "queue": "default",
    "priority": "normal",
    "jobFramework": "PyTorchJob",
    "name": "llama2-70b-pd-quickstart-v1",
    "jobSpec": {
        "command": "bash /workspace/AIAK-Training-LLM/examples/llama3/pretrain/preprocess_data.sh",
        "image": "registry.baidubce.com/aihc-aiak/aiak-training-llm:ubuntu22.04-cu12.3-torch2.2.0-py310-bccl1.2.7.2_v2.1.1.5_release",
        "replicas": 1,
        "envs": [
            {
                "name": "TOKENIZER_PATH",
                "value": "/root/pfs/models/llama3-70b/modelscope.cn/models/LLM-Research/Meta-Llama-3-70B"
            },
                        {
                "name": "input_data",
                "value": "/root/pfs/datasets/WuDaoCorpus2.0_base_sample.jsonl"
            },
                        {
                "name": "output_prefix",
                "value": "/root/pfs/datasets/WuDaoCorpus2.0_base_sample.jsonl"
            }
        ]
    },
    "datasources": [
        {
            "type": "pfs",
            "name": "pfs-oYQuh4",
            "mountPath": "/root/pfs"
        }
    ]
}
res = aihc_client.create_aijob(
    client_token=client_token,
    resourcePoolId=resourcePoolId,
    payload=payload
)
print('res', res)


## 步骤三：使用AIAK提交训练任务

### 预训练模型

In [ ]:
# -*- coding: utf-8 -*-
import sys
import json
import os
import time
import logging
from baidubce.services.aihc.aihc_client import AIHCClient

import baidubce.protocol
from baidubce.bce_client_configuration import BceClientConfiguration
from baidubce.auth.bce_credentials import BceCredentials

from dotenv import load_dotenv
import os

# 加载.env文件
load_dotenv()

# 获取配置信息, 从环境变量中获取,需要先在.env文件中设置环境变量
ak = os.getenv("AK")
sk = os.getenv("SK")
host=os.getenv("HOST")

print(ak)
print(sk)

client_token = 'test-aihc-' + str(int(time.time()))
logging.info('client_token: %s', client_token)

config = BceClientConfiguration(
    credentials=BceCredentials(ak, sk),
    endpoint=host,
    protocol=baidubce.protocol.HTTPS
)

aihc_client = AIHCClient(config)

# 创建任务
resourcePoolId = 'your resourcePoolId'
payload = {
    "queue": "default",
    "priority": "normal",
    "jobFramework": "PyTorchJob",
    "name": "llama2-70b-train-quickstart-v1",
    "jobSpec": {
        "command": "bash /workspace/AIAK-Training-LLM/examples/llama2/pretrain/pretrain_llama2_70b.sh",
        "image": "registry.baidubce.com/aihc-aiak/aiak-training-llm:ubuntu22.04-cu12.3-torch2.2.0-py310-bccl1.2.7.2_v2.1.1.5_release",
        "replicas": 4,
        "resources": [
            {
                "name": "baidu.com/a800_80g_cgpu",
                "quantity": 8
            }
        ],
        "enableRDMA": true,
        "envs": [
            {
                "name": "CUDA_DEVICE_MAX_CONNECTIONS",
                "value": "1"
            },
            {
                "name": "DATA_PATH",
                "value": "/root/pfs/datasets/WuDaoCorpus2.0_base_sample.jsonl_text_document"
            },
            {
                "name": "TOKENIZER_PATH",
                "value": "/root/pfs/models/llama3-70b/modelscope.cn/models/LLM-Research/Meta-Llama-3-70B"
            },
            {
                "name": "CHECKPOINT_PATH",
                "value": "/root/pfs/models/llama3-70b/mcore/modelscope.cn/models/LLM-Research/Meta-Llama-3-70B/tp4_pp4"
            }
        ]
    },
    "datasources": [
        {
            "type": "pfs",
            "name": "pfs-oYQuh4",
            "mountPath": "/mnt/cluster"
        }
    ]
}
res = aihc_client.create_aijob(
    client_token=client_token,
    resourcePoolId=resourcePoolId,
    payload=payload
)
pring('res', res)

## 步骤四：模型格式转换
您可以将训练获得的Megatron格式的模型转换为Huggingface格式，具体操作步骤如下。后续您可以使用转换后的Huggingface格式的模型进行服务在线部署。

## 步骤五：部署及调用模型服务
完成离线推理并评估完成模型效果后，您可以将转换为Huggingface格式的模型部署为在线服务，并在实际的生产环境中调用，从而进行推理实践。具体操作步骤如下：